In [125]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yfinance as yf
import nsepython as nse
import datetime 
import time
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error



In [126]:
# Inputs
# Date input
from_date = datetime.date(year=2015,month=1,day=1).strftime("%Y-%m-%d")
to_date = datetime.date(year=2025,month=8,day=6).strftime("%Y-%m-%d")
print(datetime.datetime.now(),' : Date inputs taken!')


2025-08-06 14:03:36.781664  : Date inputs taken!


In [127]:
df = yf.download('^NSEI',start=from_date,end=to_date)
df.columns = ['CLOSE','HIGH','LOW','OPEN','VOLUME']
df.drop('VOLUME',axis=1,inplace=True)
df['Date'] = df.index
print(datetime.datetime.now(),' : Data loaded and featured configured!')


/tmp/ipykernel_26991/3552187783.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('^NSEI',start=from_date,end=to_date)
[*********************100%***********************]  1 of 1 completed

2025-08-06 14:03:36.816269  : Data loaded and featured configured!


In [ ]:
# Additional feature generation 
for k in [5,30]:
    for j in ['OPEN','HIGH','LOW','CLOSE']:
        minl = []
        maxl = []
        for i in range(df.shape[0]):
            minimum = df[j].iloc[i:i+k].min()
            maximum = df[j].iloc[i:i+k].max()
            minl.append(minimum)
            maxl.append(maximum)
        df[f'{j}_{k}daylow'] = minl
        df[f'{j}_{k}dayhigh'] = maxl

df['CLOSE'] = df['CLOSE'].shift(-1)
df['NXT_OPEN'] = df['OPEN'].shift(-1)
print(datetime.datetime.now(),' : Additional features generated!')

2025-08-06 14:03:38.472373  : Additional features generated!


In [129]:
# Date feature dropped
df = df.drop('Date',axis=1)
print(datetime.datetime.now(),' : date feature dropped!')  


2025-08-06 14:03:38.478331  : date feature dropped!


In [ ]:
# Generate X_sample for prediction
X_sample = df.iloc[-1:,1:]
Today_open = input(f"Input opening price on {to_date} : ")
X_sample.iloc[:,-1] = Today_open

df.dropna(inplace=True)
print(datetime.datetime.now(),' : X_sample generated!')  

/tmp/ipykernel_26991/516314891.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '24641' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_sample.iloc[:,-1] = Today_open


In [131]:
# train test data generation
X_train,X_test,y_train,y_test = train_test_split(df.drop('CLOSE',axis=1),df['CLOSE'],
                                                 test_size=0.2,random_state=11)
print(datetime.datetime.now(),' : Train/test data generated!')  


2025-08-06 14:03:53.071824  : Train/test data generated!


In [132]:
# Modeling
model = XGBRegressor()
print(datetime.datetime.now(),' : XGBoost model intialised!')  


2025-08-06 14:03:53.076849  : XGBoost model intialised!


In [133]:
# Model fitting
model.fit(X_train,y_train)
print(datetime.datetime.now(),' : Model fitted on train data!')  


2025-08-06 14:03:53.361768  : Model fitted on train data!


In [134]:
# Prediction
y_pred = model.predict(X_test)
print(datetime.datetime.now(),' : Predictions done for test data!') 


2025-08-06 14:03:53.370267  : Predictions done for test data!


In [135]:
# RMSE calculation
rmse = root_mean_squared_error(y_test,y_pred)
print(datetime.datetime.now(),' : RMSE performance metric calculated!') 


2025-08-06 14:03:53.378606  : RMSE performance metric calculated!


In [136]:
# Prediction table for test data
pred_table = pd.DataFrame()
pred_table['Actual'] = y_test
pred_table['Predicted'] = y_pred
pred_table['ABSError'] = abs(pred_table['Actual']-pred_table['Predicted'])
print(datetime.datetime.now(),' : Prediction table generated!') 



print(pred_table)

2025-08-06 14:03:53.385413  : Prediction table generated!
                  Actual     Predicted    ABSError
Date                                              
2021-02-03  14895.650391  14821.229492   74.420898
2015-03-09   8712.049805   8674.476562   37.573242
2019-03-06  11058.200195  11173.227539  115.027344
2015-02-11   8711.549805   8650.059570   61.490234
2020-10-16  11873.049805  11860.406250   12.643555
...                  ...           ...         ...
2015-07-14   8523.799805   8562.703125   38.903320
2022-10-04  17331.800781  17247.236328   84.564453
2017-10-27  10363.650391  10372.416016    8.765625
2020-07-31  10891.599609  11019.770508  128.170898
2016-12-09   8170.799805   8216.548828   45.749023

[522 rows x 3 columns]


In [137]:
avg_error = pred_table['ABSError'].mean()
print(f'On an average predictions are off by +/- {int(avg_error)} points')

On an average predictions are off by +/- 92 points


In [138]:
model.predict(np.array(X_sample))

array([24532.549], dtype=float32)

In [139]:
np.array(X_sample)model.feature_names_in_


SyntaxError: invalid syntax (3627563391.py, line 1)